In [1]:
'''
Important paths
'''

bg_img_path = '/srv/PASCAL3D+_release1.1/Images/bus_imagenet/'
bg_anno_path = '/srv/PASCAL3D+_release1.1/Annotations/bus_imagenet/'
occluder_path = '/srv/occluder_libs_test_small.npz'

In [2]:
'''
Get a bounding box 
Takes the path to a .mat annotation file
'''

import scipy.io as sio

def get_properties(path):
    data = sio.loadmat(path)
    width = data['record']['size'][0][0][0][0][0]
    height = data['record']['size'][0][0][0][0][1]
    bbox = data['record'][0, 0]['objects'][0, 0]['bbox'][0].astype(int)
    return width[0, 0], height[0, 0], bbox

In [3]:
'''
get overlap percentage
'''
def overlap_ratio(occluder_bb, occludee_bb):

    #top left and bottom right points
    occluder_x1, occluder_y1, occluder_x2, occluder_y2 = occluder_bb
    occludee_x1, occludee_y1, occludee_x2, occludee_y2 = occludee_bb

    # area of the foreground object
    occludee_area = (occludee_x2 - occludee_x1) * (occludee_y2 - occludee_y1)

    # area of the background object being covered by the foreground object
    overlap_area = max(0, min(occludee_x2, occluder_x2) - max(occludee_x1, occluder_x1)) * max(0, min(occludee_y2, occluder_y2) - max(occludee_y1, occluder_y1))

    # overlap over the total background object area
    return overlap_area / occludee_area

In [4]:
import random

'''
Get a list of randomly chosen bounding boxes to occlude the background object above some threshold
This can be improved if we know the foreground image has to be some base scale to allow for above threshold occlusion
'''
def get_bbox_list(bg_bbox, bg_w, bg_h, fg_w, fg_h):

    bboxes = []

    occludee_x1 = bg_bbox[0]
    occludee_y1 = bg_bbox[1] 
    occludee_x2 = bg_bbox[2]
    occludee_y2 = bg_bbox[3]

    num_boxes = 20 # iterate over multiple boxes
    overlap_threshold = .1 # ensure a minimum amount of overlap

    # print(bg_bbox, fg_w, fg_h)
    
    for _ in range(num_boxes):
        # top left corner of the occluder bounding box:

        occluder_x1 = random.randint(max(0, occludee_x1 - fg_w), occludee_x2) # overlapping in the x-direction
        occluder_y1 = random.randint(max(0, occludee_y1 - fg_h), occludee_y2) # overlapping in the y-direction

        # TODO: Fix so doesnt exceed background image
        
        occluder_x2 = occluder_x1 + fg_w
        occluder_y2 = occluder_y1 + fg_h

        # Maybe this will fix sizing error??? (IT DOES)
        if occluder_x2 > bg_w or occluder_y2 > bg_h:
            continue

        occluder_bb = [occluder_x1, occluder_y1, occluder_x2, occluder_y2]
        occluded_ratio = overlap_ratio(occluder_bb, bg_bbox)

        if occluded_ratio >= overlap_threshold:
            bboxes.append([occluder_x1, occluder_y1, occluder_x2, occluder_y2])
    
    return bboxes

In [5]:
'''
returns the score, composite image, and compositive mask. 
num scales is the number of different foreground scales to try.
'''
from libcom import OPAScoreModel

def get_optimal_location(fg_img, fg_mask, bg_img, bg_w, bg_h, bg_bbox, num_scales):

    net = OPAScoreModel(device=0, model_type='SimOPA')
    cache_dir = './cache'

    # from libcom.fopa_heat_map.source.prepare_multi_fg_scales import prepare_multi_fg_scales
    scaled_fg_dir, scaled_mask_dir, csv_path = prepare_multi_fg_scales(cache_dir, fg_img, fg_mask, bg_img, 16)

    score = 0
    optimal_bbox = None
    best_fg = None
    best_mask = None
    best_comp = None 
    best_comp_mask = None
    
    # iterate over the different foreground scales
    with open(csv_path, mode='r', newline='') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            fg_name   = '{}_{}_{}_{}.jpg'.format(row["fg_name"].split(".")[0],row["bg_name"].split(".")[0],int(row["newWidth"]),int(row["newHeight"]))
            mask_name = '{}_{}_{}_{}.jpg'.format(row["fg_name"].split(".")[0],row["bg_name"].split(".")[0],int(row["newWidth"]),int(row["newHeight"]))
            scale     = row['scale']
            fg_w = int(row['newWidth'])
            fg_h = int(row['newHeight'])
            
            save_name = fg_name.split(".")[0] + '_' + str(scale) + '.jpg'
            bg_img    = read_image_pil(bg_img)
            fg_img    = read_image_pil(os.path.join(scaled_fg_dir, fg_name))
            fg_mask   = read_mask_pil(os.path.join(scaled_mask_dir, mask_name))
            bbox_list = get_bbox_list(bg_bbox, bg_w, bg_h, fg_w, fg_h)

            for bbox in bbox_list:
                comp, comp_mask = get_composite_image(os.path.join(scaled_fg_dir, fg_name), os.path.join(scaled_mask_dir, mask_name), bg_img, bbox)
                bbox_score = net(comp, comp_mask)
                if bbox_score > score:
                    best_fg = os.path.join(scaled_fg_dir, fg_name)
                    best_mask = os.path.join(scaled_mask_dir, mask_name)
                    optimal_bbox = bbox
                    best_comp = comp
                    best_comp_mask = comp_mask
                    score = bbox_score

        return score, best_fg, best_mask, optimal_bbox, best_comp, best_comp_mask

No module 'xformers'. Proceeding without it.


/opt/conda/envs/Libcom/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v2.0.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(


In [6]:
import random
import numpy as np
from PIL import Image
import cv2

'''
Get a random occluder and correspondiong mask
'''
def get_occluder():
    data = np.load(occluder_path, allow_pickle=True)
    
    i = random.randint(0, len(data['images']))
    print(i)
    occ_img = f'./occluders/fg_img_{i}.jpg'
    occ_mask = f'./occluders/fg_mask_{i}.png'

    image = data['images'][i]
    mask = data['masks'][i]
    print(len(data['images']))
    if mask.max() <= 1.0:  # Check if values are between 0 and 1
        mask = (mask * 255).astype(np.uint8)  # Scale to 0-255 and convert to uint8
    elif mask.dtype != np.uint8:  # If it's already 0-255 but not uint8
        mask = mask.astype(np.uint8)
        
    cv2.imwrite(occ_img, image)
    cv2.imwrite(occ_mask, mask)

    return occ_img, occ_mask

In [7]:
get_occluder()

168
2023


('./occluders/fg_img_168.jpg', './occluders/fg_mask_168.png')

In [8]:
from libcom import color_transfer
from libcom.utils.process_image import *
from libcom.utils.environment import *
from libcom import OPAScoreModel
from libcom import get_composite_image
from libcom.utils.process_image import make_image_grid
import cv2
import csv
from PIL import Image
from libcom.fopa_heat_map.source.prepare_multi_fg_scales import prepare_multi_fg_scales
from libcom import Mure_ObjectStitchModel
from libcom import ControlComModel

# for each scale, take the highest OPA score

net = Mure_ObjectStitchModel(device=0, sampler='plms')

for filename in os.listdir(bg_img_path)[:5]:
    bg_img = os.path.join(bg_img_path, filename)
    image_id = filename.split('.')[0]
    anno_path = os.path.join(bg_anno_path, image_id + '.mat')
    bg_w, bg_h, bg_bbox = get_properties(anno_path)
    fg_img, fg_mask = get_occluder()
    print(fg_mask, fg_img)
    score, fg_img, fg_mask, bbox, comp, comp_mask = get_optimal_location(fg_img, fg_mask, bg_img, bg_w, bg_h, bg_bbox, num_scales=16)
    if(score):
        res, show_fg_img = net(bg_img, [fg_img], [fg_mask], bbox, sample_steps=25, num_samples=3)
        bg_img   = draw_bbox_on_image(bg_img, bbox)
        grid_img = make_image_grid([bg_img, fg_img, comp, comp_mask] + [res[i] for i in range(3)])
        cv2.imwrite(f'./results/small_comp_{filename}.jpg'.format(image_id), grid_img)

1710
2023
./occluders/fg_mask_1710.png ./occluders/fg_img_1710.jpg
564
2023
./occluders/fg_mask_564.png ./occluders/fg_img_564.jpg
564
2023
./occluders/fg_mask_564.png ./occluders/fg_img_564.jpg
564
2023
./occluders/fg_mask_564.png ./occluders/fg_img_564.jpg
564
2023
./occluders/fg_mask_564.png ./occluders/fg_img_564.jpg
